###                               Business Case :
    
    we have given a problem on the consumer grievances against Banks  
    
-Objective:
    create an analytical and modelling framework to predict the rating of the compliance management function of Banks as 
    outstanding/satisfactory/deficient based on the given data using the above criteria.
    

In [1]:
# Import libraries
import nltk
import numpy as np
import pandas as pd
import os
import re
import string 
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn import metrics 
from nltk.corpus import stopwords
from nltk.corpus import RegexpTokenizer as regextoken
import matplotlib.pyplot as plt

C:\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
import os
os.getcwd()

'C:\\Users\\Sravanthi vejandla\\Desktop\\insofe_materials'

In [2]:
Train_data = pd.read_csv("GrievancesData_Train.csv")#### load files
df=pd.read_csv("Train.csv")


In [4]:
# merge two data sets by common BankID 
train=pd.merge(df,Train_data)

In [8]:
## converting in to appropriate data types
train.BankID = train.BankID.astype("category")
train.BankGrade  = train.BankGrade.astype("category")
train.State  = train.State .astype("category")
train.Disputed  = train.Disputed.astype("category")
train.LineOfBusiness      = train.LineOfBusiness.astype("category")
train.ResolutionComments  = train.ResolutionComments.astype("category")

In [6]:
train.isnull().sum() ### finding nulls

BankID                    0
BankGrade                 0
GrievanceID               0
State                   121
DateOfGrievance           0
Grievance_Category        0
GrievanceDescription      0
LineOfBusiness            0
ResolutionComments        0
Disputed                  0
DateOfResolution          0
dtype: int64

In [9]:
### converting into date format and claculating days to resolve
from datetime import datetime
train['DateOfGrievance'] = pd.to_datetime(train['DateOfGrievance'],format = '%Y-%m-%d')
train['DateOfResolution'] = pd.to_datetime(train['DateOfResolution'],format = '%Y-%m-%d')


In [16]:
train['days']=train.DateOfResolution-train.DateOfGrievance
train['days']=train['days'].dt.days


###  Text Preprocessing -

In [11]:
import re
from nltk.corpus import RegexpTokenizer as regextoken
stop = stopwords.words('english')
lmtzr = nltk.stem.WordNetLemmatizer()
table = str.maketrans({key: None for key in string.punctuation})

def text_preprocessing(text):
    tokenizer = regextoken('\w+') # regular expression to select one or more word character
    text = re.sub(r'\s+', ' ', str(text)) # substitute one or more whitespace with one whitespace
    text = re.sub(r'[\d]', '', str(text)) # substitute occuerence of single
    text = text.translate(table) # remove punctuations
    sentences = nltk.tokenize.sent_tokenize(text)
    sentence_tokens = [tokenizer.tokenize(sentence) for sentence in sentences]
    # convert word to lower case and remove words that are part of nltk stop words
    tokens = [[word.lower() for word in sent if word not in stop] for sent in sentence_tokens] 
    tokens = [[lmtzr.lemmatize(word) for word in sent] for sent in tokens] # convert words to its root using lemmatization
    flattened_list = [y for x in tokens for y in x]
    token_refined = [word for word in flattened_list if len(word) > 1] # remove words with length 1
    string = str(' '.join(token_refined)) # join contents of list to form a continuous string
    return (string)
    


In [12]:
train['GrievanceDescription'] = train['GrievanceDescription'].apply(lambda x: text_preprocessing(x))

In [17]:
train.head()

,BankID,BankGrade,GrievanceID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,days
0,Bank5298,satisfactory,GID347643,State53,2012-04-02,Taking out the loan or lease,for xxxx time life purchased car advised deale...,Consumer Loan,Closed with explanation,No,2012-04-18,16
1,Bank5298,satisfactory,GID353199,State53,2012-05-11,Taking out the loan or lease,on xxxxxxxx received vehicle xxxx live xxxx pu...,Consumer Loan,Closed with explanation,No,2012-06-07,27
2,Bank5298,satisfactory,GID353312,State53,2012-05-11,Taking out the loan or lease,on xxxxxxxx received vehicle xxxx live xxxx pu...,Consumer Loan,Closed with explanation,No,2012-06-07,27
3,Bank5298,satisfactory,GID353497,State9,2012-05-12,Managing the loan or lease,car loan bankand loan went vehicle almost year...,Consumer Loan,Closed with explanation,Yes,2012-06-04,23
4,Bank5298,satisfactory,GID357567,State14,2012-06-08,Managing the loan or lease,purchase vehicle xxxx xxxx xxxx xxxx xxxx xxxx...,Consumer Loan,Closed with explanation,Yes,2012-07-02,24


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=50, smooth_idf=True, tokenizer=None)
sklearn_representation=sklearn_tfidf.fit_transform(train['GrievanceDescription'])
#print(sklearn_representation)
#print(train.shape)

In [19]:
data_features = sklearn_representation.toarray()

In [20]:
data_features=pd.DataFrame(data_features)

In [21]:
print(data_features.shape) ##  converting text into TF-IDF 

(53680, 4591)


In [31]:
data = pd.concat([train, data_features], axis=1, sort=False) ### concatenating those tf-idf into train data

In [34]:
print(data.shape)


(53680, 4602)


In [33]:
data=data.drop(['GrievanceDescription'], axis=1) ### droping griveance description column 

In [14]:
data

,BankID,BankGrade,GrievanceID,State,DateOfGrievance,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,...,4581,4582,4583,4584,4585,4586,4587,4588,4589,4590
0,Bank5298,satisfactory,GID347643,State53,2012-04-02,Taking out the loan or lease,Consumer Loan,Closed with explanation,No,2012-04-18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Bank5298,satisfactory,GID353199,State53,2012-05-11,Taking out the loan or lease,Consumer Loan,Closed with explanation,No,2012-06-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Bank5298,satisfactory,GID353312,State53,2012-05-11,Taking out the loan or lease,Consumer Loan,Closed with explanation,No,2012-06-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bank5298,satisfactory,GID353497,State9,2012-05-12,Managing the loan or lease,Consumer Loan,Closed with explanation,Yes,2012-06-04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Bank5298,satisfactory,GID357567,State14,2012-06-08,Managing the loan or lease,Consumer Loan,Closed with explanation,Yes,2012-07-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,Bank5298,satisfactory,GID358743,State40,2012-06-15,Managing the loan or lease,Consumer Loan,Closed with explanation,No,2012-06-28,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,Bank5298,satisfactory,GID359723,State43,2012-06-22,Problems when you are unable to pay,Consumer Loan,Closed with explanation,No,2012-07-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7,Bank5298,satisfactory,GID360450,State28,2012-06-26,Managing the loan or lease,Consumer Loan,Closed with explanation,Yes,2012-07-23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
8,Bank5298,satisfactory,GID360822,State59,2012-06-29,Managing the loan or lease,Consumer Loan,Closed with explanation,No,2012-07-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
9,Bank5298,satisfactory,GID361232,State56,2012-07-01,Problems when you are unable to pay,Consumer Loan,Closed with explanation,No,2012-07-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [57]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

# Convert some variables to numeric
data["Grievance_Category"] = label_encoder.fit_transform(data["Grievance_Category"])
data["LineOfBusiness"] = label_encoder.fit_transform(data["LineOfBusiness"])
data["ResolutionComments"] = label_encoder.fit_transform(data["ResolutionComments"])
data["Disputed"] = label_encoder.fit_transform(data["Disputed"])
data["BankID"] = label_encoder.fit_transform(data["BankID"])

In [59]:
data=data.drop(['State'], axis=1) ## droping state column

In [63]:
data=data.drop(['GrievanceID'], axis=1) ## droping grevience id 

In [58]:
type(data)

pandas.core.frame.DataFrame

In [60]:
data.shape

(53680, 4599)

In [64]:
data.head()

,BankID,BankGrade,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,days,0,1,2,...,4581,4582,4583,4584,4585,4586,4587,4588,4589,4590
0,25,satisfactory,62,0,1,0,16,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25,satisfactory,62,0,1,0,27,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25,satisfactory,62,0,1,0,27,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25,satisfactory,45,0,1,1,23,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25,satisfactory,45,0,1,1,24,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
data=data.drop(['DateOfGrievance'], axis=1) ### dropped  dates as i calculated number of days and included it
data=data.drop(['DateOfResolution'], axis=1)

In [49]:
print(data.shape)

(53680, 4600)


In [65]:
### Splitting data into train & test

train1,test = train_test_split(data, test_size = 0.2) 

cols = [col for col in data.columns if col not in ["BankGrade"]]



In [66]:
train.x= train1[cols]
train.y=train1["BankGrade"]



In [67]:
test.x=test[cols]
test.y=test["BankGrade"]

C:\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
C:\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


#### Building RandomForest

In [54]:

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [68]:
rf_model = RandomForestClassifier(n_estimators=100, # Number of trees
                                  max_features=2,    # Num features considered
                                  oob_score=True)

In [69]:
rf_model.fit(train.x,train.y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [70]:
test_preds = rf_model.predict(X= test.x)

In [71]:
test_preds

array(['satisfactory', 'satisfactory', 'deficient', ..., 'satisfactory',
       'satisfactory', 'satisfactory'], dtype=object)

In [116]:
print("OOB accuracy: ")
print(rf_model.oob_score_)

OOB accuracy: 
0.7076425111773472


### Test data

In [22]:
Test_data = pd.read_csv("GrievancesData_Test.csv")
df_test=pd.read_csv("Test.csv")

In [23]:
print(Test_data.shape)

(27954, 10)


In [24]:
Test_data['GrievanceDescription'] = Test_data['GrievanceDescription'].apply(lambda x: text_preprocessing(x))### preprocessing test

In [25]:
sklearn_representation_1 = sklearn_tfidf.transform(Test_data['GrievanceDescription'])

In [26]:
sklearn_representation_1.shape

(27954, 4591)

In [27]:
data_features_1 = sklearn_representation_1.toarray()

In [28]:
data_features1=pd.DataFrame(data_features_1)

In [29]:
print(data_features1.shape)

(27954, 4591)


In [ ]:
data_t=data_t.drop(['GrievanceDescription'],axis=1)## droping grevience description as i included tfidf values 

In [75]:
data_t = pd.concat([Test_data,data_features1], axis=1, sort=False)

In [76]:
print(data_t.shape)

(27954, 4600)


In [86]:
data_t

,BankID,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,0,1,2,3,4,...,4582,4583,4584,4585,4586,4587,4588,4589,4590,days
0,Bank5334,"Loan servicing, payments, escrow account",Mortgage,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,11
1,Bank5278,Problems when you are unable to pay,Consumer Loan,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,15
2,Bank5372,"Loan servicing, payments, escrow account",Mortgage,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,11
3,Bank5372,"Loan servicing, payments, escrow account",Mortgage,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,11
4,Bank5310,Settlement process and costs,Mortgage,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,11
5,Bank5259,Advertising and marketing,Credit card,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,11
6,Bank5278,Identity theft / Fraud / Embezzlement,Credit card,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,52
7,Bank5373,Can't repay my loan,Student loan,Closed with explanation,Yes,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,22
8,Bank5284,"Loan modification,collection,foreclosure",Mortgage,Closed with explanation,Yes,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,22
9,Bank5316,Dealing with my lender or servicer,Student loan,Closed with explanation,No,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.059801,0.0,0.0,0.000000,0.0,0.0,37


In [81]:
data_t['DateOfGrievance'] = pd.to_datetime(data_t['DateOfGrievance'],format = '%Y-%m-%d')
data_t['DateOfResolution'] = pd.to_datetime(data_t['DateOfResolution'],format = '%Y-%m-%d')

data_t['days']=data_t.DateOfResolution-data_t.DateOfGrievance
data_t['days']=data_t['days'].dt.days


In [87]:
data_t.shape

(27954, 4597)

In [123]:
data_t['days']=data_t['days'].dt.days

In [83]:
data_t=data_t.drop(['DateOfGrievance'], axis=1)
data_t=data_t.drop(['DateOfResolution'], axis=1)

In [84]:
data_t=data_t.drop(['State'], axis=1)

In [85]:
data_t=data_t.drop(['GrievanceID'], axis=1)

In [88]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

# Convert some variables to numeric
data_t["Grievance_Category"] = label_encoder.fit_transform(data_t["Grievance_Category"])
data_t["LineOfBusiness"] = label_encoder.fit_transform(data_t["LineOfBusiness"])
data_t["ResolutionComments"] = label_encoder.fit_transform(data_t["ResolutionComments"])
data_t["Disputed"] = label_encoder.fit_transform(data_t["Disputed"])

In [89]:
data_t["BankID"] = label_encoder.fit_transform(data_t["BankID"])

In [90]:
preds = rf_model.predict(X= data_t)

In [91]:
len(preds)

27954

In [93]:
predicted=pd.DataFrame(preds,columns=['BankGrade'])

In [94]:
test_bankid1=Test_data['BankID'].tolist()

In [95]:
grade=predicted['BankGrade'].tolist() 

In [96]:
sub=pd.DataFrame({'BankGrade' :grade,'BankID' :test_bankid1},index=None,columns=['BankID', 'BankGrade'])

In [97]:
data=sub.groupby('BankID',as_index=False)

In [98]:
b=pd.DataFrame(data.first())

In [99]:
b

,BankID,BankGrade
0,Bank5246,satisfactory
1,Bank5259,deficient
2,Bank5271,satisfactory
3,Bank5272,satisfactory
4,Bank5278,satisfactory
5,Bank5284,satisfactory
6,Bank5299,satisfactory
7,Bank5310,satisfactory
8,Bank5312,satisfactory
9,Bank5316,satisfactory


In [100]:
b.to_csv('submission_rf.csv',index=False)

### Building logistic regression

In [101]:

from sklearn.linear_model import LogisticRegression


In [102]:
mul_lr =LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(train.x, train.y)
    
print(metrics.accuracy_score(train.y, mul_lr.predict(train.x)))
print(metrics.accuracy_score(test.y, mul_lr.predict(test.x)))
 

0.8404899403874814
0.7984351713859911


In [103]:
len(test.y)

10736

In [104]:
u= mul_lr.predict(data_t)

In [105]:
u

array(['satisfactory', 'deficient', 'satisfactory', ..., 'satisfactory',
       'deficient', 'deficient'], dtype=object)

In [108]:
predicted1=pd.DataFrame(u,columns=['BankGrade'])

In [109]:
grade1=predicted1['BankGrade'].tolist() 

In [110]:
sub=pd.DataFrame({'BankGrade' :grade1,'BankID' :test_bankid1},index=None,columns=['BankID', 'BankGrade'])

In [111]:
data1=sub.groupby('BankID',as_index=False)

In [112]:
c=pd.DataFrame(data1.first())

In [117]:
c.to_csv('submission_lr.csv',index=False)

###    conclusion:
 I build three model on the given data,Depending on the analysis Banks can come to know which factors are influencing && improve their profits